In [1]:
#!wget https://raw.githubusercontent.com/jthorton/MSM_QCArchive/master/msm.offxml

In [2]:
from openff.toolkit.typing.engines.smirnoff import ForceField, vdWHandler, AngleHandler, BondHandler
from openff.toolkit.typing.engines.smirnoff.parameters import LibraryChargeHandler

In [3]:
msm_ff = ForceField('msm.offxml', allow_cosmetic_attributes=True)
sage_rc1 = ForceField('openff_unconstrained-2.0.0-rc.1.offxml', allow_cosmetic_attributes=True)
hybrid_ff = ForceField('openff_unconstrained-1.3.1.offxml', allow_cosmetic_attributes=True)

## Creating a hybrid FF with bond and angle parameters from modified seminario, torsion parameters from 1.3.1, and vdW parameters from sage rc1

In [4]:
hybrid_ff.deregister_parameter_handler('Bonds')
bond_handler = BondHandler(version=0.3)
for parameter in msm_ff['Bonds'].parameters:
    bond_handler.add_parameter(parameter.to_dict())
hybrid_ff.register_parameter_handler(bond_handler)

In [5]:
hybrid_ff.deregister_parameter_handler('Angles')
angle_handler = AngleHandler(version=0.3)
for parameter in msm_ff['Angles'].parameters:
    angle_handler.add_parameter(parameter.to_dict())
hybrid_ff.register_parameter_handler(angle_handler)

In [6]:
hybrid_ff.deregister_parameter_handler('vdW')
vdw_handler = vdWHandler(version=0.3)
for parameter in sage_rc1['vdW'].parameters:
    vdw_handler.add_parameter(parameter.to_dict())
hybrid_ff.register_parameter_handler(vdw_handler)

In [7]:
hybrid_ff.deregister_parameter_handler('LibraryCharges')
librarycharges_handler = LibraryChargeHandler(version=0.3)
for parameter in sage_rc1['LibraryCharges'].parameters:
    librarycharges_handler.add_parameter(parameter.to_dict())
hybrid_ff.register_parameter_handler(librarycharges_handler)

In [8]:
for ptype in ['Angles', 'Bonds', 'ProperTorsions']:
    for i in range(len(hybrid_ff.get_parameter_handler(ptype).parameters)):
        item = hybrid_ff.get_parameter_handler(ptype).parameters[i]
        dd = item.to_dict()
        if dd['id'] in ['t156', 't157', 't158']:
            continue
        dd['parameterize'] = []
        for key, value in dd.items():
            if key.startswith('k') or key.startswith('angle') or key.startswith('length'):
                dd['parameterize'].append(key)
        dd['parameterize'] = ','.join(dd['parameterize'])
        hybrid_ff.get_parameter_handler(ptype).parameters[i].add_cosmetic_attribute('parameterize', dd['parameterize'])

In [9]:
hybrid_ff.to_file('hybrid_msm_sagerc1.offxml')